In [ ]:
import pandas as pd
def extract_classification(file_path):
    classification = []
    df = pd.read_csv(file_path)
    classification = df['tissue_types'].tolist()
    classification = list(set(classification))
    return classification

csv_path = '/mnt/e/Temp_data/Body_Parts_XRay/Original/train_df_new1.csv'
classification = extract_classification(csv_path)

classification

In [ ]:
#classification

import torch, os
from PIL import Image
import numpy as np
import open_clip

pretrained = '/mnt/g/Logtemp/open_clip/Whale_Dolphin_Identification/2023_08_18-21_14_32-model_coca_ViT-L-14-lr_1e-06-b_32-j_4-p_amp/checkpoints/epoch_16.pt'
# model, _, preprocess = open_clip.create_model_and_transforms('coca_ViT-L-14', pretrained='mscoco_finetuned_laion2B-s13B-b90k')
model, _, preprocess = open_clip.create_model_and_transforms(
  model_name="coca_ViT-L-14",
  pretrained=pretrained, #mscoco_finetuned_laion2B-s13B-b90k
)

tokenizer = open_clip.get_tokenizer('coca_ViT-L-14')

image_path = '/mnt/g/Datasets/Whale_Dolphin_Identification/Square/test/images/false_killer_whale_bdea86a4d11fa9.jpg'
img = Image.open(image_path).convert('RGB')
img = img.resize((224, 224), Image.Resampling.LANCZOS)
image = preprocess(img).unsqueeze(0)
text = tokenizer(species)

filename = os.path.splitext(os.path.basename(image_path))[0]
ground_truth = "_".join(filename.split('_')[:-1]).replace("_", " ")

with torch.no_grad(), torch.cuda.amp.autocast():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

    # Construct the dictionary
    class_predict_dict = dict(zip(species, text_probs))
    # Extract the key with the largest value
    class_predict = max(class_predict_dict, key=class_predict_dict.get)
    
    if ground_truth == class_predict:
        predicted_correct = 1
    else:
        predicted_correct = 0

print("Label probs:", text_probs)  # prints: [[1., 0., 0.]]
print("Ground truth:", ground_truth)  # prints: 0
print("Predicted class:", class_predict)  # prints: 0
print("Predicted correct:", predicted_correct)  # prints: 1

In [1]:
# Batch test for Direct Classification benchmark 1
import torch, os, open_clip
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm

def extract_classification(file_path):
    classification = []
    df_classification = pd.read_csv(file_path)
    classification = df_classification['tissue_types'].tolist()
    classification = list(set(classification))
    return classification, df_classification

classification_csv_file = '/mnt/g/Datasets/Body_Parts_XRay/train_df_new1.csv'
classification, df_classification = extract_classification(classification_csv_file)


def process_single_epoch(pretrained_model, sentences):
    img_dir = '/mnt/g/Datasets/Body_Parts_XRay/Original/test/images/'
    model, _, preprocess = open_clip.create_model_and_transforms(
        model_name="coca_ViT-L-14",
        pretrained=pretrained_model,
    )
    pt_dir = os.path.dirname(pretrained_model)
    epoch_number = os.path.basename(pretrained_model).split('epoch_')[1].split('.pt')[0]
    df_all = pd.DataFrame()

    # Prepare the tokenizer for sentences
    tokenizer = open_clip.get_tokenizer('coca_ViT-L-14')
    text = tokenizer(sentences)
    with torch.no_grad(), torch.cuda.amp.autocast():
        text_features = model.encode_text(text)
        text_features /= text_features.norm(dim=-1, keepdim=True)

    def read_single_image(img_dir, img_filename, sentences, text_features):
        #get the ground truth value
        image_path = os.path.join(img_dir, img_filename)
        ground_truth = df_classification[df_classification['filename']==img_filename]['tissue_types'].iloc[0]
        
        # load a sample image
        image = preprocess(Image.open(image_path)).unsqueeze(0)
        with torch.no_grad(), torch.cuda.amp.autocast():
            image_features = model.encode_image(image)
            image_features /= image_features.norm(dim=-1, keepdim=True)
            text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
            text_probs = text_probs.cpu().tolist()[0]

        # Construct the dictionary
        class_predict_dict = dict(zip(sentences, text_probs))
        # Extract the key with the largest value
        class_predict = max(class_predict_dict, key=class_predict_dict.get)
        
        if ground_truth == class_predict:
            predicted_correct = 1
        else:
            predicted_correct = 0
        
        pred_dict = {'img_filename': [img_filename], 'class_predict_dict': [class_predict_dict], 'ground_truth': [ground_truth], 'class_predict': [class_predict], 'predicted_correct': [predicted_correct]}
        df_single = pd.DataFrame(pred_dict)
        
        return df_single

    all_images = os.listdir(img_dir)
    for img_filename in tqdm(all_images, desc=f"epoch_{epoch_number}: Processing images", unit="image"):
        df = read_single_image(img_dir, img_filename, sentences, text_features)
        df_all = pd.concat([df_all, df]).reset_index(drop=True)

    #save df_all to csv
    csv_file = f"temp/epoch_{epoch_number}.csv"
    df_all.to_csv(csv_file, index=False)

    total_rows = len(df_all)
    # Count the occurrences of '1' in the 'predicted_correct' column
    count_ones = df_all['predicted_correct'].sum()
    epoch_filename = os.path.splitext(os.path.basename(pretrained_model))[0]
    predicted_correct_pct = count_ones /total_rows *100
    print(f"Epoch_file: {epoch_filename}. Total images: {total_rows}, predicted_correct': {count_ones}, predicted_correct (%)': {predicted_correct_pct}%")
    
    epoch_dict = {'pt_dir': [pt_dir],
                'epoch_filename': [epoch_filename], 
                'sentences': [sentences], 
                'total_rows': [total_rows], 
                'predicted_correct': [count_ones], 
                'predicted_correct (%)': [predicted_correct_pct]}
    df_single_epoch = pd.DataFrame(epoch_dict)
    return df_single_epoch


def process_single_pretrained_model(pretrained_model, csv_file):
    df_single_pretrained_model = pd.DataFrame()
    df = process_single_epoch(pretrained_model, classification)
    df_single_pretrained_model = pd.concat([df_single_pretrained_model, df]).reset_index(drop=True)
    save_to_csv(df_single_pretrained_model, csv_file)
    
def extract_epoch_num(filepath):
    # Split the filename from the path
    filename = os.path.basename(filepath) #filepath.split('/')[-1]
    # Extract the number between "epoch_" and ".pt"
    epoch_num = int(filename.split('epoch_')[1].split('.pt')[0])
    return epoch_num

def save_to_csv(df, csv_file):
    if not os.path.exists(csv_file):
        df.to_csv(csv_file, index=False)
    else:
        # If the CSV file already exists, append without header
        df.to_csv(csv_file, mode='a', header=False, index=False)

def main():
    pt_folder_path = '/mnt/g/Logtemp/open_clip/Body_Parts_XRay/2023_08_20-12_05_28-model_coca_ViT-L-14-lr_1e-06-b_32-j_4-p_amp/checkpoints/'
    pt_files = [os.path.join(pt_folder_path, filename) for filename in os.listdir(pt_folder_path) if filename.endswith('.pt')]
    #filter pt_files with low epoch number
    pt_files = [pt_file for pt_file in pt_files if extract_epoch_num(pt_file) >= 0]
    # Sort the list
    sorted_pt_files = sorted(pt_files, key=extract_epoch_num)
    csv_file = "benchmark_all_epochs.csv"
    for pretrained_model in sorted_pt_files:
        process_single_pretrained_model(pretrained_model, csv_file)


main()

epoch_1: Processing images: 100%|██████████| 174/174 [01:33<00:00,  1.86image/s]


Epoch_file: epoch_1. Total images: 174, predicted_correct': 20, predicted_correct (%)': 11.494252873563218%


epoch_2: Processing images: 100%|██████████| 174/174 [01:30<00:00,  1.93image/s]


Epoch_file: epoch_2. Total images: 174, predicted_correct': 20, predicted_correct (%)': 11.494252873563218%


epoch_3: Processing images: 100%|██████████| 174/174 [01:30<00:00,  1.92image/s]


...

Epoch_file: epoch_97. Total images: 174, predicted_correct': 149, predicted_correct (%)': 85.63218390804597%


epoch_98: Processing images: 100%|██████████| 174/174 [01:28<00:00,  1.97image/s]


Epoch_file: epoch_98. Total images: 174, predicted_correct': 149, predicted_correct (%)': 85.63218390804597%


epoch_99: Processing images: 100%|██████████| 174/174 [01:28<00:00,  1.96image/s]

Epoch_file: epoch_99. Total images: 174, predicted_correct': 146, predicted_correct (%)': 83.9080459770115%


In [1]:
# Batch test for Direct Classification benchmark 2 (cropped square images)
import torch, os, open_clip
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm

def extract_classification(file_path):
    classification = []
    df_classification = pd.read_csv(file_path)
    classification = df_classification['tissue_types'].tolist()
    classification = list(set(classification))
    return classification, df_classification

classification_csv_file = '/mnt/g/Datasets/Body_Parts_XRay/train_df_new1.csv'
classification, df_classification = extract_classification(classification_csv_file)


def process_single_epoch(pretrained_model, sentences):
    img_dir = '/mnt/g/Datasets/Body_Parts_XRay/Square/test/images/'
    model, _, preprocess = open_clip.create_model_and_transforms(
        model_name="coca_ViT-L-14",
        pretrained=pretrained_model,
    )
    pt_dir = os.path.dirname(pretrained_model)
    epoch_number = os.path.basename(pretrained_model).split('epoch_')[1].split('.pt')[0]
    df_all = pd.DataFrame()

    # Prepare the tokenizer for sentences
    tokenizer = open_clip.get_tokenizer('coca_ViT-L-14')
    text = tokenizer(sentences)
    with torch.no_grad(), torch.cuda.amp.autocast():
        text_features = model.encode_text(text)
        text_features /= text_features.norm(dim=-1, keepdim=True)

    def read_single_image(img_dir, img_filename, sentences, text_features):
        #get the ground truth value
        image_path = os.path.join(img_dir, img_filename)
        ground_truth = df_classification[df_classification['filename']==img_filename]['tissue_types'].iloc[0]
        
        # load a sample image
        image = preprocess(Image.open(image_path)).unsqueeze(0)
        with torch.no_grad(), torch.cuda.amp.autocast():
            image_features = model.encode_image(image)
            image_features /= image_features.norm(dim=-1, keepdim=True)
            text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
            text_probs = text_probs.cpu().tolist()[0]

        # Construct the dictionary
        class_predict_dict = dict(zip(sentences, text_probs))
        # Extract the key with the largest value
        class_predict = max(class_predict_dict, key=class_predict_dict.get)
        
        if ground_truth == class_predict:
            predicted_correct = 1
        else:
            predicted_correct = 0
        
        pred_dict = {'img_filename': [img_filename], 'class_predict_dict': [class_predict_dict], 'ground_truth': [ground_truth], 'class_predict': [class_predict], 'predicted_correct': [predicted_correct]}
        df_single = pd.DataFrame(pred_dict)
        
        return df_single

    all_images = os.listdir(img_dir)
    for img_filename in tqdm(all_images, desc=f"epoch_{epoch_number}: Processing images", unit="image"):
        df = read_single_image(img_dir, img_filename, sentences, text_features)
        df_all = pd.concat([df_all, df]).reset_index(drop=True)

    #save df_all to csv
    csv_file = f"temp/epoch_{epoch_number}.csv"
    df_all.to_csv(csv_file, index=False)

    total_rows = len(df_all)
    # Count the occurrences of '1' in the 'predicted_correct' column
    count_ones = df_all['predicted_correct'].sum()
    epoch_filename = os.path.splitext(os.path.basename(pretrained_model))[0]
    predicted_correct_pct = count_ones /total_rows *100
    print(f"Epoch_file: {epoch_filename}. Total images: {total_rows}, predicted_correct': {count_ones}, predicted_correct (%)': {predicted_correct_pct}%")
    
    epoch_dict = {'pt_dir': [pt_dir],
                'epoch_filename': [epoch_filename], 
                'sentences': [sentences], 
                'total_rows': [total_rows], 
                'predicted_correct': [count_ones], 
                'predicted_correct (%)': [predicted_correct_pct]}
    df_single_epoch = pd.DataFrame(epoch_dict)
    return df_single_epoch


def process_single_pretrained_model(pretrained_model, csv_file):
    df_single_pretrained_model = pd.DataFrame()
    df = process_single_epoch(pretrained_model, classification)
    df_single_pretrained_model = pd.concat([df_single_pretrained_model, df]).reset_index(drop=True)
    save_to_csv(df_single_pretrained_model, csv_file)
    
def extract_epoch_num(filepath):
    # Split the filename from the path
    filename = os.path.basename(filepath) #filepath.split('/')[-1]
    # Extract the number between "epoch_" and ".pt"
    epoch_num = int(filename.split('epoch_')[1].split('.pt')[0])
    return epoch_num

def save_to_csv(df, csv_file):
    if not os.path.exists(csv_file):
        df.to_csv(csv_file, index=False)
    else:
        # If the CSV file already exists, append without header
        df.to_csv(csv_file, mode='a', header=False, index=False)

def main():
    pt_folder_path = '/mnt/g/Logtemp/open_clip/Body_Parts_XRay/2023_08_21-19_57_50-model_coca_ViT-L-14-lr_1e-06-b_32-j_4-p_amp/checkpoints/'
    pt_files = [os.path.join(pt_folder_path, filename) for filename in os.listdir(pt_folder_path) if filename.endswith('.pt')]
    #filter pt_files with low epoch number
    pt_files = [pt_file for pt_file in pt_files if extract_epoch_num(pt_file) >= 68]
    # Sort the list
    sorted_pt_files = sorted(pt_files, key=extract_epoch_num)
    csv_file = "benchmark_all_epochs.csv"
    for pretrained_model in sorted_pt_files:
        process_single_pretrained_model(pretrained_model, csv_file)


main()

epoch_2: Processing images: 100%|██████████| 174/174 [01:25<00:00,  2.05image/s]


Epoch_file: epoch_2. Total images: 174, predicted_correct': 23, predicted_correct (%)': 13.218390804597702%


epoch_4: Processing images: 100%|██████████| 174/174 [01:39<00:00,  1.75image/s]


Epoch_file: epoch_4. Total images: 174, predicted_correct': 25, predicted_correct (%)': 14.367816091954023%


epoch_6: Processing images: 100%|██████████| 174/174 [01:40<00:00,  1.73image/s]


...

Epoch_file: epoch_94. Total images: 174, predicted_correct': 152, predicted_correct (%)': 87.35632183908046%


epoch_96: Processing images: 100%|██████████| 174/174 [01:47<00:00,  1.62image/s]


Epoch_file: epoch_96. Total images: 174, predicted_correct': 152, predicted_correct (%)': 87.35632183908046%


epoch_98: Processing images: 100%|██████████| 174/174 [01:41<00:00,  1.72image/s]

Epoch_file: epoch_98. Total images: 174, predicted_correct': 145, predicted_correct (%)': 83.33333333333334%


In [2]:
# Batch test for Direct Classification benchmark 3 (cropped square & data augmentation images)
import torch, os, open_clip
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm
from torchvision.transforms import Normalize, ToTensor

def _convert_to_rgb(image):
    return image.convert('RGB')
mean = (0.3453, 0.3453, 0.3453)
std = (0.2566, 0.2566, 0.2566)
to_tensor = ToTensor()
normalize = Normalize(mean=mean, std=std)

def extract_classification(file_path):
    classification = []
    df_classification = pd.read_csv(file_path)
    classification = df_classification['tissue_types'].tolist()
    classification = list(set(classification))
    return classification, df_classification

classification_csv_file = '/mnt/g/Datasets/Body_Parts_XRay/train_df_new1.csv'
classification, df_classification = extract_classification(classification_csv_file)


def process_single_epoch(pretrained_model, sentences):
    img_dir = '/mnt/g/Datasets/Body_Parts_XRay/Square/test/images/'
    model, _, preprocess = open_clip.create_model_and_transforms(
        model_name="coca_ViT-L-14",
        pretrained=pretrained_model,
    )
    pt_dir = os.path.dirname(pretrained_model)
    epoch_number = os.path.basename(pretrained_model).split('epoch_')[1].split('.pt')[0]
    df_all = pd.DataFrame()

    # Prepare the tokenizer for sentences
    tokenizer = open_clip.get_tokenizer('coca_ViT-L-14')
    text = tokenizer(sentences)
    with torch.no_grad(), torch.cuda.amp.autocast():
        text_features = model.encode_text(text)
        text_features /= text_features.norm(dim=-1, keepdim=True)

    def read_single_image(img_dir, img_filename, sentences, text_features):
        #get the ground truth value
        image_path = os.path.join(img_dir, img_filename)
        ground_truth = df_classification[df_classification['filename']==img_filename]['tissue_types'].iloc[0]
        
        # load a sample image
        image = preprocess(Image.open(image_path)) #.unsqueeze(0)
        image = _convert_to_rgb(image)
        image = to_tensor(image)
        image = normalize(image)
        image = image.unsqueeze(0)
        with torch.no_grad(), torch.cuda.amp.autocast():
            image_features = model.encode_image(image)
            image_features /= image_features.norm(dim=-1, keepdim=True)
            text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
            text_probs = text_probs.cpu().tolist()[0]

        # Construct the dictionary
        class_predict_dict = dict(zip(sentences, text_probs))
        # Extract the key with the largest value
        class_predict = max(class_predict_dict, key=class_predict_dict.get)
        
        if ground_truth == class_predict:
            predicted_correct = 1
        else:
            predicted_correct = 0
        
        pred_dict = {'img_filename': [img_filename], 'class_predict_dict': [class_predict_dict], 'ground_truth': [ground_truth], 'class_predict': [class_predict], 'predicted_correct': [predicted_correct]}
        df_single = pd.DataFrame(pred_dict)
        
        return df_single

    all_images = os.listdir(img_dir)
    for img_filename in tqdm(all_images, desc=f"epoch_{epoch_number}: Processing images", unit="image"):
        df = read_single_image(img_dir, img_filename, sentences, text_features)
        df_all = pd.concat([df_all, df]).reset_index(drop=True)

    #save df_all to csv
    csv_file = f"temp/epoch_{epoch_number}.csv"
    df_all.to_csv(csv_file, index=False)

    total_rows = len(df_all)
    # Count the occurrences of '1' in the 'predicted_correct' column
    count_ones = df_all['predicted_correct'].sum()
    epoch_filename = os.path.splitext(os.path.basename(pretrained_model))[0]
    predicted_correct_pct = count_ones /total_rows *100
    print(f"Epoch_file: {epoch_filename}. Total images: {total_rows}, predicted_correct': {count_ones}, predicted_correct (%)': {predicted_correct_pct}%")
    
    epoch_dict = {'pt_dir': [pt_dir],
                'epoch_filename': [epoch_filename], 
                'sentences': [sentences], 
                'total_rows': [total_rows], 
                'predicted_correct': [count_ones], 
                'predicted_correct (%)': [predicted_correct_pct]}
    df_single_epoch = pd.DataFrame(epoch_dict)
    return df_single_epoch


def process_single_pretrained_model(pretrained_model, csv_file):
    df_single_pretrained_model = pd.DataFrame()
    df = process_single_epoch(pretrained_model, classification)
    df_single_pretrained_model = pd.concat([df_single_pretrained_model, df]).reset_index(drop=True)
    save_to_csv(df_single_pretrained_model, csv_file)
    
def extract_epoch_num(filepath):
    # Split the filename from the path
    filename = os.path.basename(filepath) #filepath.split('/')[-1]
    # Extract the number between "epoch_" and ".pt"
    epoch_num = int(filename.split('epoch_')[1].split('.pt')[0])
    return epoch_num

def save_to_csv(df, csv_file):
    if not os.path.exists(csv_file):
        df.to_csv(csv_file, index=False)
    else:
        # If the CSV file already exists, append without header
        df.to_csv(csv_file, mode='a', header=False, index=False)

def main():
    pt_folder_path = '/mnt/g/Logtemp/open_clip/Body_Parts_XRay/2023_08_21-23_03_36-model_coca_ViT-L-14-lr_1e-06-b_32-j_4-p_amp/checkpoints/'
    pt_files = [os.path.join(pt_folder_path, filename) for filename in os.listdir(pt_folder_path) if filename.endswith('.pt')]
    #filter pt_files with low epoch number
    pt_files = [pt_file for pt_file in pt_files if extract_epoch_num(pt_file) >= 0]
    # Sort the list
    sorted_pt_files = sorted(pt_files, key=extract_epoch_num)
    csv_file = "benchmark_all_epochs_augment1.csv"
    for pretrained_model in sorted_pt_files:
        process_single_pretrained_model(pretrained_model, csv_file)


main()

epoch_2: Processing images: 100%|██████████| 174/174 [01:44<00:00,  1.67image/s]


Epoch_file: epoch_2. Total images: 174, predicted_correct': 23, predicted_correct (%)': 13.218390804597702%


epoch_4: Processing images: 100%|██████████| 174/174 [01:35<00:00,  1.82image/s]


Epoch_file: epoch_4. Total images: 174, predicted_correct': 23, predicted_correct (%)': 13.218390804597702%


epoch_6: Processing images: 100%|██████████| 174/174 [01:34<00:00,  1.84image/s]


...

Epoch_file: epoch_196. Total images: 174, predicted_correct': 154, predicted_correct (%)': 88.50574712643679%


epoch_198: Processing images: 100%|██████████| 174/174 [01:32<00:00,  1.88image/s]


Epoch_file: epoch_198. Total images: 174, predicted_correct': 154, predicted_correct (%)': 88.50574712643679%


epoch_200: Processing images: 100%|██████████| 174/174 [01:28<00:00,  1.96image/s]

Epoch_file: epoch_200. Total images: 174, predicted_correct': 154, predicted_correct (%)': 88.50574712643679%


In [8]:
# Batch test for Direct Classification benchmark 4 (cropped square & data augmentation images, normalizing with mean and std)
import torch, os, open_clip
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm
from torchvision.transforms import ToTensor
import albumentations as A
from PIL import Image


class AlbumentationsTransform2:
    def __init__(self, transform=None):
        self.transform = transform

    def __call__(self, img):
        img = np.array(img)  # Convert to numpy array
        img = self.transform(image=img)['image']
        return Image.fromarray(img)  # Convert back to PIL image

# Define the CLAHE transformation
clahe = A.CLAHE(p=1.0, clip_limit=4.0, tile_grid_size=(8, 8))
CLAHE = AlbumentationsTransform2(clahe), # normalizing using Adaptive Histogram Equalization (CLAHE)

def _convert_to_rgb(image):
    return image.convert('RGB')
to_tensor = ToTensor()


def extract_classification(file_path):
    classification = []
    df_classification = pd.read_csv(file_path)
    classification = df_classification['tissue_types'].tolist()
    classification = list(set(classification))
    return classification, df_classification

classification_csv_file = '/mnt/g/Datasets/Body_Parts_XRay/train_df_new1.csv'
classification, df_classification = extract_classification(classification_csv_file)


def process_single_epoch(pretrained_model, sentences):
    img_dir = '/mnt/g/Datasets/Body_Parts_XRay/Square/test/images/'
    model, _, preprocess = open_clip.create_model_and_transforms(
        model_name="coca_ViT-L-14",
        pretrained=pretrained_model,
    )
    pt_dir = os.path.dirname(pretrained_model)
    epoch_number = os.path.basename(pretrained_model).split('epoch_')[1].split('.pt')[0]
    df_all = pd.DataFrame()

    # Prepare the tokenizer for sentences
    tokenizer = open_clip.get_tokenizer('coca_ViT-L-14')
    text = tokenizer(sentences)
    with torch.no_grad(), torch.cuda.amp.autocast():
        text_features = model.encode_text(text)
        text_features /= text_features.norm(dim=-1, keepdim=True)

    def read_single_image(img_dir, img_filename, sentences, text_features):
        #get the ground truth value
        image_path = os.path.join(img_dir, img_filename)
        ground_truth = df_classification[df_classification['filename']==img_filename]['tissue_types'].iloc[0]
        
        # load a sample image
        image = preprocess(Image.open(image_path).convert('RGB')) #.unsqueeze(0)
        # image = to_tensor(image)
        image = normalize(image)
        image = image.unsqueeze(0)

        with torch.no_grad(), torch.cuda.amp.autocast():
            image_features = model.encode_image(image)
            image_features /= image_features.norm(dim=-1, keepdim=True)
            text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
            text_probs = text_probs.cpu().tolist()[0]

        # Construct the dictionary
        class_predict_dict = dict(zip(sentences, text_probs))
        # Extract the key with the largest value
        class_predict = max(class_predict_dict, key=class_predict_dict.get)
        
        if ground_truth == class_predict:
            predicted_correct = 1
        else:
            predicted_correct = 0
        
        pred_dict = {'img_filename': [img_filename], 'class_predict_dict': [class_predict_dict], 'ground_truth': [ground_truth], 'class_predict': [class_predict], 'predicted_correct': [predicted_correct]}
        df_single = pd.DataFrame(pred_dict)
        
        return df_single

    all_images = os.listdir(img_dir)
    for img_filename in tqdm(all_images, desc=f"epoch_{epoch_number}: Processing images", unit="image"):
        df = read_single_image(img_dir, img_filename, sentences, text_features)
        df_all = pd.concat([df_all, df]).reset_index(drop=True)

    #save df_all to csv
    csv_file = f"temp/epoch_{epoch_number}.csv"
    df_all.to_csv(csv_file, index=False)

    total_rows = len(df_all)
    # Count the occurrences of '1' in the 'predicted_correct' column
    count_ones = df_all['predicted_correct'].sum()
    epoch_filename = os.path.splitext(os.path.basename(pretrained_model))[0]
    predicted_correct_pct = count_ones /total_rows *100
    print(f"Epoch_file: {epoch_filename}. Total images: {total_rows}, predicted_correct': {count_ones}, predicted_correct (%)': {predicted_correct_pct}%")
    
    epoch_dict = {'pt_dir': [pt_dir],
                'epoch_filename': [epoch_filename], 
                'sentences': [sentences], 
                'total_rows': [total_rows], 
                'predicted_correct': [count_ones], 
                'predicted_correct (%)': [predicted_correct_pct]}
    df_single_epoch = pd.DataFrame(epoch_dict)
    return df_single_epoch


def process_single_pretrained_model(pretrained_model, csv_file):
    df_single_pretrained_model = pd.DataFrame()
    df = process_single_epoch(pretrained_model, classification)
    df_single_pretrained_model = pd.concat([df_single_pretrained_model, df]).reset_index(drop=True)
    save_to_csv(df_single_pretrained_model, csv_file)
    
def extract_epoch_num(filepath):
    # Split the filename from the path
    filename = os.path.basename(filepath) #filepath.split('/')[-1]
    # Extract the number between "epoch_" and ".pt"
    epoch_num = int(filename.split('epoch_')[1].split('.pt')[0])
    return epoch_num

def save_to_csv(df, csv_file):
    if not os.path.exists(csv_file):
        df.to_csv(csv_file, index=False)
    else:
        # If the CSV file already exists, append without header
        df.to_csv(csv_file, mode='a', header=False, index=False)

def main():
    pt_folder_path = '/mnt/g/Logtemp/open_clip/Body_Parts_XRay/2023_08_21-23_03_36-model_coca_ViT-L-14-lr_1e-06-b_32-j_4-p_amp/checkpoints/'
    pt_files = [os.path.join(pt_folder_path, filename) for filename in os.listdir(pt_folder_path) if filename.endswith('.pt')]
    #filter pt_files with low epoch number
    pt_files = [pt_file for pt_file in pt_files if extract_epoch_num(pt_file) >= 0]
    # Sort the list
    sorted_pt_files = sorted(pt_files, key=extract_epoch_num)
    csv_file = "benchmark_all_epochs_augment3.csv"
    for pretrained_model in sorted_pt_files:
        process_single_pretrained_model(pretrained_model, csv_file)


main()

epoch_2: Processing images: 100%|██████████| 174/174 [01:57<00:00,  1.48image/s]


Epoch_file: epoch_2. Total images: 174, predicted_correct': 33, predicted_correct (%)': 18.96551724137931%


epoch_4: Processing images: 100%|██████████| 174/174 [01:55<00:00,  1.50image/s]


Epoch_file: epoch_4. Total images: 174, predicted_correct': 32, predicted_correct (%)': 18.39080459770115%


epoch_6: Processing images: 100%|██████████| 174/174 [01:50<00:00,  1.58image/s]


...

epoch_198: Processing images: 100%|██████████| 174/174 [01:32<00:00,  1.89image/s]


Epoch_file: epoch_198. Total images: 174, predicted_correct': 139, predicted_correct (%)': 79.88505747126436%


epoch_200: Processing images: 100%|██████████| 174/174 [01:31<00:00,  1.90image/s]

Epoch_file: epoch_200. Total images: 174, predicted_correct': 139, predicted_correct (%)': 79.88505747126436%


: 

In [3]:
import os
pt_folder_root = '/mnt/g/Logtemp/open_clip/Body_Parts_XRay/'
dir_1 = [a for a in os.listdir(pt_folder_root) if a.startswith('2023_08_22')][0]
os.path.join(pt_folder_root, dir_1, 'checkpoints')

'/mnt/g/Logtemp/open_clip/Body_Parts_XRay/2023_08_22-07_36_22-model_coca_ViT-L-14-lr_1e-06-b_32-j_4-p_amp/checkpoints'